import libraries and dataset

In [2]:
import pandas as pd
import numpy as np

In [3]:
file_path = r"C:\Users\Tanishq\Documents\stuttgart\Study\winter 25\DS and AI for industry work1\project\data_science_project\ted_main_v2.csv\ted_main_v2.csv"

In [4]:
import os
print(os.path.exists(file_path))

True


In [5]:
df = pd.read_csv(file_path)

In [6]:
df.head(10)

,speaker_name,title,posted_date,duration,Link,about_speaker,about_talk,views,tags
0,Alex Gendler,The Egyptian myth of the death of Osiris,Jul 2020,3:56,https://www.ted.com/talks/alex_gendler_the_egy...,NaN,"Long jealous of his older brother Osiris, the ...","208,703","education,ancient world,TED-Ed"
1,Shari Davis,What if you could help decide how the governme...,Jul 2020,10:28,https://www.ted.com/talks/shari_davis_what_if_...,As a leader of the Participatory Budgeting Pro...,What if you could help decide how the governme...,"425,688","democracy,leadership,community"
2,Nita Mosby Tyler,Want a more just world? Be an unlikely ally,Jul 2020,10:15,https://www.ted.com/talks/nita_mosby_tyler_wan...,Nita Mosby Tyler specializes in the developmen...,A more equal world starts with you. Citing a f...,"460,269","activism,inequality,race"
3,Susan Lupack,The race to decode a mysterious language,Jul 2020,4:24,https://www.ted.com/talks/susan_lupack_the_rac...,NaN,"In the early 1900s, archaeologist Sir Arthur E...","350,202","TED-Ed,education,language"
4,Ariel Waldman,The colorful critter world of microbes in Anta...,Jul 2020,5:56,https://www.ted.com/talks/ariel_waldman_the_co...,"An artist who's pivoted to science, Ariel Wald...","In this tour of the microscopic world, explore...","333,482","science,animals,exploration"
5,Thomas Abt,Why violence clusters in cities — and how to r...,Jul 2020,14:51,https://www.ted.com/talks/thomas_abt_why_viole...,"Thomas Abt developed an innovative, evidence-i...",Reducing violence in cities in the US isn't th...,"50,273","violence,United States,crime"
6,Steven Anderson,What makes volcanoes erupt?,Jul 2020,5:10,https://www.ted.com/talks/steven_anderson_what...,NaN,"In February of 1942, Mexican farmer Dionisio P...","169,272","TED-Ed,education,animation"
7,Christina Agapakis,What happens when biology becomes technology?,Jul 2020,10:58,https://www.ted.com/talks/christina_agapakis_w...,"In her art and scientific work, Christina Agap...","""We've been promised a future of chrome -- but...","575,657","science,biology,nature"
8,Ahmad M. Hasnah,Rethinking education and celebrating the Arabi...,Jul 2020,26:32,https://www.ted.com/talks/ahmad_m_hasnah_rethi...,"Qatar Foundation for Education, Science, and C...",The pandemic has deepened the need to rethink ...,NaN,"education,society,future"
9,Rod Phillips,What happened when the United States tried to ...,Jul 2020,4:54,https://www.ted.com/talks/rod_phillips_what_ha...,NaN,"On January 17, 1920, less than one hour after ...","262,067","TED-Ed,animation,government"


In [7]:
df.columns

Index(['speaker_name', 'title', 'posted_date', 'duration', 'Link',
       'about_speaker', 'about_talk', 'views', 'tags'],
      dtype='object')